# 对希拉里克林顿的邮件进行LDA主题提取

In [2]:
import pandas as pd
df = pd.read_csv('data/HillaryEmails.csv')
df.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [17]:
mailContent = df['ExtractedBodyText']
mailContent.dropna(inplace=True)
import re
def textPreProcess(text):
    text = re.sub(r'\n',' ',text) #去除换行
    text = re.sub(r'\d+/\d+/\d+','',text) #去除日期
    text = re.sub(r'[0-2]?[0-9]:[0-5][0-9]','',text) #去除时间
    text = re.sub(r'[\w]+@[\.\w]+','',text) #去除邮件地址
    text = re.sub(r'/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i', '', text) # 去除网址
    text = re.sub(r'[^\w ]','',text)
    pure_text = ''
    #去除其他特殊字符
    for letter in text:
        if letter.isalpha() or letter == ' ':
            pure_text+=letter.lower()
    text = ' '.join(word for word in pure_text.split() if len(word) > 1)
    return text
docs=mailContent.apply(textPreProcess)

In [18]:
docs.head(1).values

array(['thursday march pm latest how syria is aiding qaddafi and more sid hrc memo syria aiding libya docx hrc memo syria aiding libya docx march for hillary'],
      dtype=object)

In [20]:
docsList = docs.values
docsList[0]

'thursday march pm latest how syria is aiding qaddafi and more sid hrc memo syria aiding libya docx hrc memo syria aiding libya docx march for hillary'

In [22]:
from nltk.corpus import stopwords
from gensim import corpora,models,similarities
import gensim
stopwordsList = stopwords.words('english')
texts = [[word for word in doc.split() if word not in stopwordsList] for doc in docs]
texts[0]

['thursday',
 'march',
 'pm',
 'latest',
 'syria',
 'aiding',
 'qaddafi',
 'sid',
 'hrc',
 'memo',
 'syria',
 'aiding',
 'libya',
 'docx',
 'hrc',
 'memo',
 'syria',
 'aiding',
 'libya',
 'docx',
 'march',
 'hillary']

In [24]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
ldaModel = models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary,num_topics=10)

In [25]:
ldaModel.print_topics(num_topics=20,num_words=10)

[(0,
  '0.016*"doc" + 0.014*"state" + 0.010*"negotiating" + 0.008*"release" + 0.008*"part" + 0.006*"house" + 0.006*"us" + 0.005*"department" + 0.005*"lona" + 0.005*"press"'),
 (1,
  '0.024*"fyi" + 0.007*"israeli" + 0.005*"bloomberg" + 0.004*"would" + 0.004*"know" + 0.004*"new" + 0.004*"like" + 0.004*"israel" + 0.004*"last" + 0.004*"us"'),
 (2,
  '0.005*"state" + 0.005*"us" + 0.003*"eg" + 0.003*"new" + 0.003*"office" + 0.003*"would" + 0.003*"party" + 0.003*"said" + 0.003*"worse" + 0.003*"email"'),
 (3,
  '0.076*"pm" + 0.031*"office" + 0.024*"secretarys" + 0.019*"meeting" + 0.017*"room" + 0.013*"state" + 0.011*"department" + 0.010*"route" + 0.010*"arrive" + 0.010*"en"'),
 (4,
  '0.012*"percent" + 0.009*"ok" + 0.006*"sent" + 0.006*"see" + 0.005*"sbwhoeop" + 0.005*"got" + 0.005*"poll" + 0.005*"voters" + 0.005*"ill" + 0.005*"time"'),
 (5,
  '0.007*"us" + 0.006*"obama" + 0.006*"new" + 0.006*"would" + 0.006*"said" + 0.006*"one" + 0.005*"president" + 0.005*"american" + 0.004*"mr" + 0.003*"gove

In [29]:
#预测新文档属于分类的哪一类
predText = 'To Barack and Michelle Obama, our country owes you an enormous debt of gratitude. We thank you for your graceful, determined leadership'
pred = [word for word in predText.lower().split() if word not in stopwordsList]
predBow = dictionary.doc2bow(pred)
ldaModel.get_document_topics(predBow)

[(0, 0.0100923665),
 (1, 0.17156367),
 (2, 0.01009353),
 (3, 0.010092159),
 (4, 0.010093549),
 (5, 0.0100955255),
 (6, 0.7476888),
 (7, 0.010095637),
 (8, 0.010092945),
 (9, 0.010091894)]

In [31]:
dictionary.token2id['obama']

311

In [35]:
dictionary.id2token[311]

'obama'